In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

In [2]:
url = "http://yustiks.ru/dataset/cardio_train.csv"
data = pd.read_csv(url,sep=";")
data.tail()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1
69999,99999,20540,1,170,72.0,120,80,2,1,0,0,1,0


In [3]:
y = data.iloc[:,-1].values
X = data.iloc[:,1:-2].values
f_names = data.columns[1:-2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
sc = MinMaxScaler(feature_range=(-1,1))
X_train_norm = sc.fit_transform(X_train)
X_test_norm = sc.transform(X_test)

In [5]:
model = RandomForestClassifier(n_estimators=100, n_jobs = 4)
model.fit(X_train_norm,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [6]:
model_pred = model.predict(X_test_norm)

print(classification_report(y_test, model_pred))
# model.score(X_test_norm, y_test)

              precision    recall  f1-score   support

           0       0.71      0.72      0.72      7083
           1       0.71      0.70      0.71      6917

    accuracy                           0.71     14000
   macro avg       0.71      0.71      0.71     14000
weighted avg       0.71      0.71      0.71     14000



0.7117857142857142

In [7]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

print("Feature ranking:")
for f in range(X.shape[1]):
    print(f"{f+1}. feature '{f_names[f]}' ({importances[indices[f]]})")

Feature ranking:
1. feature 'age' (0.3227200021400742)
2. feature 'gender' (0.17536719837608322)
3. feature 'height' (0.17186292809169398)
4. feature 'weight' (0.15357006863617131)
5. feature 'ap_hi' (0.09110142678026027)
6. feature 'ap_lo' (0.03678651271184033)
7. feature 'cholesterol' (0.016824600567447102)
8. feature 'gluc' (0.01562516069717583)
9. feature 'smoke' (0.008786653891136394)
10. feature 'alco' (0.007355448108117419)
